In [1]:
from astropy.io import ascii
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import math

# read in file downloaded from SDSS
sdss_file = ascii.read('N_rich_QSOs_w_offsets.csv',format='csv')
 
# I had stupid naming convention in output file
# ra, dec: of bright offset star
# ra_deg, dec_deg: of QSO
offset_coords = SkyCoord(ra=sdss_file["ra"],dec=sdss_file["dec"],unit=(u.deg,u.deg))
qso_coords = SkyCoord(ra=sdss_file["ra_deg"],dec=sdss_file["dec_deg"],unit=(u.deg,u.deg))

### Below is a slightly-changed version of your for-loop from before.  It includes both a fix for your original problem *and* some other minor changes I included just to help me understand what was going in

In [2]:
%%time
# only keep 1 offset star, the one that's closest
unique_srcs, unique_inds = np.unique(sdss_file["name"],return_index=True)

offset_rmag_keep = []

# set up arrays to hold sexagesimal RA and Dec of offset star
offset_ra_out  = []
offset_dec_out = []

offset_ra_out_deg  = []
offset_dec_out_deg = []

# set up arrays to hold offsets in RA and Dec
del_ra  = []
del_dec = []

# cycle through each source
for src, qso_coord in zip(unique_srcs, qso_coords[unique_inds]):
    
    # find the offset stars associated with this star
    match = sdss_file["name"] == src
    
    # find the closest offset star
    dis  = qso_coord.separation(offset_coords[match]).arcsecond
    keep = dis == min(dis)
    
    # THIS IS THE MAGIC TO SOLVE THE PROBLEM:
    assert np.sum(keep) == 1 # this isn't strictly necessary, but is a sanity-check to make sure there's exactly one "keep" match
    # the line below is the trick: before you were creating lists where each element was an array of length-one, because
    # the "keep" part yields an *array* rather than a single value.  But by grabbing just the one value we make it into
    # a single value and everything is happy
    matchkeep_coord = offset_coords[match][keep][0]
    
    # calculate offset from offset star TO target
    dra,ddec = matchkeep_coord.spherical_offsets_to(qso_coord)
    
    del_ra.append(np.round(dra.to(u.arcsec)/u.arcsec,1))
    del_dec.append(np.round(ddec.to(u.arcsec)/u.arcsec,1))
    
    # for output file, save RA and Dec of preferred offset star in sexagesimal format    
    offset_ra_out.append(matchkeep_coord.ra.to_string(u.hourangle,sep=':',pad=True,precision=2))
    offset_dec_out.append(matchkeep_coord.dec.to_string(u.degree,sep=':',pad=True,precision=1))
    
    offset_rmag_keep.append(sdss_file["r"][match][keep])

CPU times: user 7.17 s, sys: 9.66 ms, total: 7.18 s
Wall time: 7.18 s


### this is a different take that uses *only* array operations, no for-loops - I recommend this, as it's at ~50x faster and a bit shorter/possibly easier to read

In [3]:
%%time

# only keep 1 offset star, the one that's closest
unique_srcs, unique_inds = np.unique(sdss_file["name"], return_index=True)

idx, d2d, _ = qso_coords[unique_inds].match_to_catalog_sky(offset_coords[unique_inds])
# idx is now the index into offset_coords[unique_inds] that finds the *closest* match between the two

# I *might* have your offset direction backwards here?
dra, ddec = offset_coords[unique_inds][idx].spherical_offsets_to(qso_coords[unique_inds])

# the next four lines you could just as well put directly in the following cell, but I put it here so you can see
# how they map onto what you did the for-loop way
offset_ra_out = offset_coords[unique_inds].ra.to_string(u.hourangle,sep=':',pad=True,precision=2)
offset_dec_out = offset_coords[unique_inds].dec.to_string(u.degree,sep=':',pad=True,precision=1)
del_ra = np.round(dra.arcsec, 1)
del_dec = np.round(ddec.arcsec, 1)

CPU times: user 115 ms, sys: 10.4 ms, total: 125 ms
Wall time: 125 ms


In [4]:
towrite = sdss_file[unique_inds]['name', 'r', 'imag', 'comment']

towrite['Targ_RA'] = qso_coords.ra[unique_inds].to_string(u.hourangle,sep=':',pad=True,precision=2)
towrite['Targ_Dec'] = qso_coords.dec[unique_inds].to_string(u.deg,sep=':',pad=True,precision=1)
towrite['Offset_RA'] = offset_ra_out
towrite['Offset_Dec'] = offset_dec_out
towrite['del_RA'] = del_ra
towrite['del_Dec'] = del_dec

In [5]:
# this just re-names the columns and puts them in a different order. Not strictly necessary but makes it look a bit
# more like your version below
r = towrite['r']
del towrite['r']
towrite['r_mag'] = r

i = towrite['imag']
del towrite['imag']
towrite['i_mag'] = i

c = towrite['comment']
del towrite['comment']
towrite['comment'] = c

towrite

name,Targ_RA,Targ_Dec,Offset_RA,Offset_Dec,del_RA,del_Dec,r_mag,i_mag,comment
str19,str11,str11,str11,str11,float64,float64,float64,float64,str10
J000118.40-010221.8,00:01:18.40,-01:02:21.8,00:01:17.90,-01:02:46.3,7.4,24.5,16.82182,19.38,N-rich_QSO
J000759.40+150822.6,00:07:59.40,15:08:22.6,00:08:00.94,15:08:34.4,-22.3,-11.8,13.5828,18.84,N-rich_QSO
J003815.92+140304.5,00:38:15.92,14:03:04.5,00:38:19.36,14:02:52.7,-50.0,11.8,13.65194,19.66,N-rich_QSO
J004600.26-002122.4,00:46:00.26,-00:21:22.4,00:45:57.43,-00:22:18.8,42.4,56.4,15.82157,19.81,N-rich_QSO
J010119.85+145635.2,01:01:19.85,14:56:35.2,01:01:15.08,14:56:59.6,69.1,-24.4,14.50011,18.68,N-rich_QSO
J012530.85-102739.8,01:25:30.85,-10:27:39.8,01:25:26.83,-10:26:38.6,59.2,-61.2,15.67185,18.02,N-rich_QSO
J013724.43-082419.9,01:37:24.43,-08:24:19.9,01:37:27.03,-08:24:26.7,-38.6,6.7,14.64875,18.44,N-rich_QSO
J014504.35+125214.1,01:45:04.35,12:52:14.1,01:45:09.15,12:51:09.0,-70.2,65.1,16.88905,19.47,N-rich_QSO
J014517.82+135602.2,01:45:17.82,13:56:02.2,01:45:20.85,13:55:56.4,-44.1,5.8,16.34576,18.75,N-rich_QSO


In [6]:
towrite.write('new_version.txt', format='ascii.csv')
!cat new_version.txt  #this shows you the actual content of the output file in the cell output

name,Targ_RA,Targ_Dec,Offset_RA,Offset_Dec,del_RA,del_Dec,r_mag,i_mag,comment
J000118.40-010221.8,00:01:18.40,-01:02:21.8,00:01:17.90,-01:02:46.3,7.4,24.5,16.82182,19.38,N-rich_QSO
J000759.40+150822.6,00:07:59.40,15:08:22.6,00:08:00.94,15:08:34.4,-22.3,-11.8,13.5828,18.84,N-rich_QSO
J003815.92+140304.5,00:38:15.92,14:03:04.5,00:38:19.36,14:02:52.7,-50.0,11.8,13.65194,19.66,N-rich_QSO
J004600.26-002122.4,00:46:00.26,-00:21:22.4,00:45:57.43,-00:22:18.8,42.4,56.4,15.82157,19.81,N-rich_QSO
J010119.85+145635.2,01:01:19.85,14:56:35.2,01:01:15.08,14:56:59.6,69.1,-24.4,14.50011,18.68,N-rich_QSO
J012530.85-102739.8,01:25:30.85,-10:27:39.8,01:25:26.83,-10:26:38.6,59.2,-61.2,15.67185,18.02,N-rich_QSO
J013724.43-082419.9,01:37:24.43,-08:24:19.9,01:37:27.03,-08:24:26.7,-38.6,6.7,14.64875,18.44,N-rich_QSO
J014504.35+125214.1,01:45:04.35,12:52:14.1,01:45:09.15,12:51:09.0,-70.2,65.1,16.88905,19.47,N-rich_QSO
J014517.82+135602.2,01:45:17.82,13:56:02.2,01:45:20.85,13:55:56.4,-44.1,5.8,16.34576,18.75,N-r

One other thing: I often find it best to do rounding of the sort you're doing for dra/ddec here only at the time of writing the file.  That way you can actually use the full-precision version in the code even after you've written out the table.  The `write` method lets you do that as follows:

Also, in case you haven't seen it before, a lambda function is basically an inline function.  So ``f = lambda x:...`` used below  is equivalent to:
```
def f(x):
    return ...
```
so what we're doing here is saying that for the columns 'del_RA' and 'del_Dec', take each value, re-format into a floating-point string with a single decimal precision, and use that as the element in the output file.

In [7]:
# first put in the full-precision versions of these columns
towrite['del_RA'] = dra.arcsec
towrite['del_Dec'] = ddec.arcsec

towrite.write('new_version2.txt', format='ascii.csv', 
              formats={'del_RA': lambda x: '{:.1f}'.format(x), 
                       'del_Dec': lambda x: '{:.1f}'.format(x)})

!cat new_version2.txt

name,Targ_RA,Targ_Dec,Offset_RA,Offset_Dec,del_RA,del_Dec,r_mag,i_mag,comment
J000118.40-010221.8,00:01:18.40,-01:02:21.8,00:01:17.90,-01:02:46.3,7.4,24.5,16.82182,19.38,N-rich_QSO
J000759.40+150822.6,00:07:59.40,15:08:22.6,00:08:00.94,15:08:34.4,-22.3,-11.8,13.5828,18.84,N-rich_QSO
J003815.92+140304.5,00:38:15.92,14:03:04.5,00:38:19.36,14:02:52.7,-50.0,11.8,13.65194,19.66,N-rich_QSO
J004600.26-002122.4,00:46:00.26,-00:21:22.4,00:45:57.43,-00:22:18.8,42.4,56.4,15.82157,19.81,N-rich_QSO
J010119.85+145635.2,01:01:19.85,14:56:35.2,01:01:15.08,14:56:59.6,69.1,-24.4,14.50011,18.68,N-rich_QSO
J012530.85-102739.8,01:25:30.85,-10:27:39.8,01:25:26.83,-10:26:38.6,59.2,-61.2,15.67185,18.02,N-rich_QSO
J013724.43-082419.9,01:37:24.43,-08:24:19.9,01:37:27.03,-08:24:26.7,-38.6,6.7,14.64875,18.44,N-rich_QSO
J014504.35+125214.1,01:45:04.35,12:52:14.1,01:45:09.15,12:51:09.0,-70.2,65.1,16.88905,19.47,N-rich_QSO
J014517.82+135602.2,01:45:17.82,13:56:02.2,01:45:20.85,13:55:56.4,-44.1,5.8,16.34576,18.75,N-r